In [3]:
import os
import sys
import pathlib

sys.path.append(os.path.join(str(pathlib.Path.cwd().parent)))

## Features

In [8]:
from bofire.domain.features import ContinuousInput, DiscreteInput, CategoricalInput, CategoricalDescriptorInput

x1 = ContinuousInput(key="x1", lower_bound=0, upper_bound=1)
x2 = ContinuousInput(key="x2", lower_bound=0, upper_bound=1)
x3 = ContinuousInput(key="x3", lower_bound=0, upper_bound=1)
x4 = DiscreteInput(key="x4", values=[1, 2, 5, 7.5])
x5 = CategoricalInput(key="x5", categories=["A", "B", "C"], allowed=[True,True,False])
x6 = CategoricalDescriptorInput(key="x6", categories=["c1", "c2", "c3"], descriptors=["d1", "d2"], values = [[1,2],[2,5],[1,7]])

In [9]:
from bofire.domain.features import ContinuousOutput
from bofire.domain.objectives import MaximizeObjective, MinimizeObjective

objective1 = MaximizeObjective(
    w=1.0, 
    lower_bound=0, 
    upper_bound=1,
)
y1 = ContinuousOutput(key="y1", objective=objective1)

objective2 = MinimizeObjective(
    w=1.0
)
y2 = ContinuousOutput(key="y2", objective=objective2)

In [10]:
from bofire.domain.features import InputFeatures, OutputFeatures

input_features = InputFeatures(features = [x1, x2, x3, x4, x5, x6])
output_features = OutputFeatures(features=[y1, y2])

In [23]:
x5 = input_features.get_by_key('x5')
x5

CategoricalInput(key='x5', type='CategoricalInput', categories=['A', 'B', 'C'], allowed=[True, True, False])

In [26]:
input_features.get_keys()

['x1', 'x2', 'x3', 'x4', 'x6', 'x5']

In [27]:
output_features.get_keys()

['y1', 'y2']

In [28]:
free_inputs = input_features.get_free()
fixed_inputs = input_features.get_fixed()

InputFeatures(features=[ContinuousInput(key='x1', type='ContinuousInput', lower_bound=0.0, upper_bound=1.0), ContinuousInput(key='x2', type='ContinuousInput', lower_bound=0.0, upper_bound=1.0), ContinuousInput(key='x3', type='ContinuousInput', lower_bound=0.0, upper_bound=1.0), DiscreteInput(key='x4', type='DiscreteInput', values=[1.0, 2.0, 5.0, 7.5]), CategoricalInput(key='x5', type='CategoricalInput', categories=['A', 'B', 'C'], allowed=[True, True, False]), CategoricalDescriptorInput(key='x6', type='CategoricalDescriptorInput', categories=['c1', 'c2', 'c3'], allowed=[True, True, True], descriptors=['d1', 'd2'], values=[[1.0, 2.0], [2.0, 5.0], [1.0, 7.0]])])

In [32]:
samples_x5 = x5.sample(2)
samples_x5

0    B
1    B
Name: x5, dtype: object


In [29]:
X = input_features.sample(n=10)
X

,x1,x2,x3,x4,x6,x5
0,0.333488,0.916704,0.662074,7.5,c3,B
1,0.461435,0.828921,0.972609,2.0,c3,B
2,0.979737,0.750465,0.911565,1.0,c3,B
3,0.029544,0.855302,0.505408,5.0,c2,A
4,0.335937,0.737333,0.339590,1.0,c2,B
5,0.911805,0.274892,0.879965,2.0,c3,B
6,0.995641,0.174091,0.168966,5.0,c1,A
7,0.957184,0.739900,0.930788,1.0,c1,A
8,0.044343,0.263328,0.425307,5.0,c2,B
9,0.177980,0.995390,0.490606,1.0,c2,B


## Constraints

In [12]:
from bofire.domain.constraints import LinearEqualityConstraint, LinearInequalityConstraint

# A mixture: x1 + x2 + x3 = 1
constr1 = LinearEqualityConstraint(features=["x1", "x2", "x3"], coefficients=[1,1,1], rhs=1)

# x1 + 2 * x3 < 0.8
constr2 = LinearInequalityConstraint(features=["x1", "x3"], coefficients=[1, 2], rhs=0.8)

In [13]:
from bofire.domain.constraints import NonlinearEqualityConstraint, NonlinearInequalityConstraint

# The unit circle: x1**2 + x2**2 = 1
constr3 = NonlinearEqualityConstraint(expression="x1**2 + x2**2 - 1")

# Require x1 < 0.5 if x5 == "A"
constr4 = NonlinearInequalityConstraint(expression="(x1 - 0.5) * (x5 =='A')")

In [15]:
from bofire.domain.constraints import NChooseKConstraint

# Only 2 or 3 out of 5 parameters can be greater than zero
constr5 = NChooseKConstraint(features=["x1", "x2", "x3", "x4", "x5"], min_count=2, max_count=3, none_also_valid=True)

In [ ]:
from bofire.domain.constraints import Constraints

constraints = Constraints(constraints=[constr1, constr2, constr3, constr4, constr5])